### parsing online data

https://kingcounty.gov/depts/health/covid-19/data/daily-summary.aspx

In [39]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [33]:
url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
df = pd.read_csv(url, index_col="date", parse_dates=True)

In [34]:
kc = df.query("county == 'King'").copy()
kc.sort_values(by="date", inplace=True)
kc.head()

,county,state,fips,cases,deaths
date,,,,,
2020-02-28,King,Washington,53033.0,1,0
2020-02-29,King,Washington,53033.0,4,1
2020-03-01,King,Washington,53033.0,11,3
2020-03-02,King,Washington,53033.0,15,6
2020-03-03,King,Washington,53033.0,22,10


In [40]:
kc

,county,state,fips,cases,deaths
date,,,,,
2020-02-28,King,Washington,53033.0,1,0
2020-02-29,King,Washington,53033.0,4,1
2020-03-01,King,Washington,53033.0,11,3
2020-03-02,King,Washington,53033.0,15,6
2020-03-03,King,Washington,53033.0,22,10
2020-03-04,King,Washington,53033.0,33,11
2020-03-05,King,Washington,53033.0,52,11
2020-03-06,King,Washington,53033.0,59,12
2020-03-07,King,Washington,53033.0,72,16
